In [1]:
# Inbuilt Python modules
import os
import time
import requests
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# Imports required for Selenium Chromedriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located as poel

# Function to get the Google search URL
def get_google_search_url(query,recency):
    query = query.replace(' ', '+')
    url = f"https://duckduckgo.com/?q={query}&ia=web"
    
    if recency != "":
        url += f"&df={recency}"
    
    return url
# defining driver
opt = Options()
opt.add_argument("--start-maximized")

# doesn't wait for the page to complete loading
caps = DesiredCapabilities.CHROME
caps["pageLoadStrategy"] = "eager"

service = Service("./chromedriver.exe")

driver = webdriver.Chrome(
    service=service, 
    options=opt, 
    desired_capabilities= caps
)

wait = WebDriverWait(driver, 3)

In [3]:
queries = [
    "cat",
    "dog"
]
# max times it will click more results
pages = 10
# can be out of "d", "w", "m", "y", ""
recency = "m" 

# Domains to avoid
avoid = [
    "amazon",
    "wikipedia",
    "imdb",
    "duckduckgo",
    "youtube",
    "google"
]

f_title = [
    "blog"
]


for query in queries:
    record = {
        "title": [],
        "link": [],
        "snippet": []
    }

    driver.get(get_google_search_url(query, recency))
    # while True:
    body = wait.until(poel(
        (By.TAG_NAME, "body")
        ))

    for i in range(pages):
        try:
            next_btn = wait.until(poel(
                (By.ID, "more-results")
                ))
            action = ActionChains(driver)
            action.move_to_element(next_btn)

            action.perform()
            next_btn.click()
        except Exception as e:
            break
        time.sleep(1)

    results = driver.find_elements(By.CSS_SELECTOR, 'li[data-layout="organic"]')

    i = 1
    for result in results:
        try:
            link_tag = result.find_element(By.CSS_SELECTOR, 'a[data-testid="result-title-a"]')
            title = link_tag.text
            link = link_tag.get_attribute("href")

            for domain in avoid:
                if domain in link:
                    continue

            try:
                snippet = result.find_element(By.CSS_SELECTOR, 'div[data-result="snippet"]').text
            except:
                snippet = ""

            if f:
                for f in f_title:
                    if (f.lower() in title.lower()) or (f.lower() in snippet.lower()):
                        record["title"].append(title)
                        record["link"].append(link)
                        record["snippet"].append(snippet)
            else:
                record["title"].append(title)
                record["link"].append(link)
                record["snippet"].append(snippet)

            i+=1
            
        except Exception as e:
            print(e)
            continue

    df = pd.DataFrame(record)
    print(f"{query} : {len(df)} results")
    df.to_csv(f"./results/{query}.csv", index=False)

cat : 52 results
dog : 48 results
